In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import os, cv2
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']=''

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from statsmodels.distributions.empirical_distribution import ECDF

import keras.backend as K
from segmentation_models import Unet
from segmentation_models.utils import set_trainable

from tgs.encoders import BinEncoder
from tgs.models import get_kernel_net, predict_lr
from tgs.losses import bce_tversky, comp_metric, lovasz_loss, comp_metric_no_sigmoid
from tgs.cyclical_learning_rate import CyclicSnapshot
from tgs.preprocessing import extend_tb, reflect_lr

Using TensorFlow backend.


In [25]:
def resize_and_pad(img):
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    img = reflect_lr(img, 11, 11)
    img = extend_tb(img, 11, 11)                            
    return img

def train_generator(df):
    
    depth_range = np.expand_dims(np.linspace(-50, 50, 101), 1)
    
    while True:
        shuffle_indices = np.arange(len(df))
        shuffle_indices = np.random.permutation(shuffle_indices)
        
        for start in range(0, len(df), batch_size):
            
            end = min(start + batch_size, len(df))            
            inds = shuffle_indices[start:end]
            images = train_images[inds]
            masks = train_masks[inds]
            
            n = len(inds)
            batch_images = np.zeros((2*n, img_size, img_size, 3), dtype=np.float32)
            batch_masks = np.zeros((2*n, 101, 101, 1), dtype=np.float32)
            for i, (img, msk, z) in enumerate(zip(images, masks, df.z[inds])):
                
                batch_images[i,:,:,1] = resize_and_pad(img)
                
                dep = 0.001*np.repeat(z+depth_range, img.shape[0], 1)     
                batch_images[i,:,:,1] = resize_and_pad(dep)
            
                ecdf_img = ECDF(img.flatten())(img)
                batch_images[i,:,:,2] = resize_and_pad(ecdf_img)
                                       
                batch_images[n+i,:,:,:] = batch_images[i,:,::-1,:]
                
                batch_masks[i,:,:,0] = msk
                batch_masks[n+i,:,:,:] =  batch_masks[i,:,::-1,:]
                
            yield batch_images, batch_masks
            
def valid_generator(df):
    
    depth_range = np.expand_dims(np.linspace(-50, 50, 101), 1)
    
    while True:
        shuffle_indices = np.arange(len(df))
        shuffle_indices = np.random.permutation(shuffle_indices)
        
        for start in range(0, len(df), batch_size):
            
            end = min(start + batch_size, len(df))            
            inds = shuffle_indices[start:end]
            images = train_images[inds]
            masks = train_masks[inds]
            
            n = len(inds)
            batch_images = np.zeros((n, img_size, img_size, 3), dtype=np.float32)
            batch_masks = np.zeros((n, 101, 101, 1), dtype=np.float32)
            for i, (img, msk, z) in enumerate(zip(images, masks, df.z[inds])):
                
                batch_images[i,:,:,1] = resize_and_pad(img)
                
                dep = 0.001*np.repeat(z+depth_range, img.shape[0], 1)     
                batch_images[i,:,:,1] = resize_and_pad(dep)
            
                ecdf_img = ECDF(img.flatten())(img)
                batch_images[i,:,:,2] = resize_and_pad(ecdf_img)
                       
                batch_masks[i,:,:,0] = msk
                
            yield batch_images, batch_masks

In [4]:
# Set some parameters
MACHINE = 'matt'
backbone = 'resnet50'
version = 'qubvel-1c'

n_folds = 10
img_size = 224
batch_size = 32
decoder_filters = (64, 32, 16, 8, 4)
cycle_len = 3
    
model_name = '_'.join([
    f'{version}',
    f'backbone-{backbone}',    
    f'decoder-{"-".join(str(x) for x in decoder_filters)}',    
    f'size-{img_size}',
    f'cycle_len-{cycle_len}',
]) 

In [5]:
# load data
FILE_PATH = f'/home/{MACHINE}/Dropbox/Kaggle/tgs'
train_df = pd.read_csv(f'{FILE_PATH}/train.csv', index_col='id', usecols=[0])
depths_df = pd.read_csv(f'{FILE_PATH}/depths.csv', index_col='id')

train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)].copy()

In [6]:
# add classes 
be = BinEncoder(n_bins=10)
test_df['class'] = be.fit_transform(test_df.z.values)

In [7]:
def load_images(path, indexs):
    images = np.zeros((len(indexs), 101, 101, 1), dtype=np.float32)
    for i, idx in enumerate(indexs):
        images[i,:,:,0] = cv2.imread(f'{path}/{idx}.png',0)/255
    return images

# load images
IMAGE_PATH = f'/media/{MACHINE}/storage0/kaggle-tgs-data'
train_images = load_images(f'{IMAGE_PATH}/test_images', test_df.index)
train_masks = np.load(f'{FILE_PATH}/blends/kernelnet-4_alpha_0.45_mask-best_test.npy').astype(np.float32)

## Define Model

In [20]:
import keras
def get_unet(final_activation, **kwargs):
    base_model = Unet(**kwargs, activation='linear')
    x = base_model.output
    x = keras.layers.Cropping2D(cropping=[[11, 11], [11, 11]])(x)
    x = keras.layers.AveragePooling2D()(x)
    y = keras.layers.Activation(final_activation)(x)
    model = keras.Model(inputs=base_model.input, outputs=y)
    return model

In [ ]:
# initialize containers
loss_list, metric_list, epoch_list, = [], [], []

# cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True)
fold_generator = kfold.split(test_df, test_df['class'])
train_idx, valid_idx = next(fold_generator)

# get step size
step_size = (cycle_len*len(train_idx))//(2*batch_size)

# pretrain model decoder
K.clear_session()
model = get_unet(final_activation='sigmoid', 
                 backbone_name=backbone,
                 input_shape=(img_size, img_size, 3),
                 encoder_weights='imagenet11k-places365ch',
                 freeze_encoder=False,
                 decoder_filters=decoder_filters,
                 decoder_use_batchnorm=True)
model.compile('sgd', bce_tversky, [comp_metric])

# create callbacks
weights_file = f'{FILE_PATH}/weights/{model_name}_stage-1'
callbacks_list = [CyclicSnapshot(
    cycle_len=cycle_len,
    base_lr=0.001, 
    max_lr=0.01,    
    mode='triangular2',
    step_size = step_size,
    filename=weights_file)]      

# initialize model
model.fit_generator(generator=train_generator(test_df.iloc[train_idx]),
                    steps_per_epoch=np.ceil(float(len(train_idx)) / float(batch_size)),
                    epochs=50,
                    verbose=2,
                    callbacks=callbacks_list,
                    validation_data=valid_generator(test_df.iloc[valid_idx]),
                    validation_steps=np.ceil(float(len(valid_idx)) / float(batch_size)))

# get new model without activation
K.clear_session()
model = get_unet(final_activation='linear', 
                 backbone_name=backbone,
                 input_shape=(img_size, img_size, 3),
                 encoder_weights=None,
                 freeze_encoder=False,
                 decoder_filters=decoder_filters,
                 decoder_use_batchnorm=True)

model.load_weights(f'{FILE_PATH}/weights/{model_name}_stage-1-5.hdf5')
model.compile('sgd', lovasz_loss, [comp_metric_no_sigmoid])

# create callbacks
weights_file = f'{FILE_PATH}/weights/{model_name}_stage-2'
callbacks_list = [CyclicSnapshot(
    cycle_len=cycle_len,
    base_lr=0.0001, 
    max_lr=0.001,    
    mode='triangular2',
    step_size = step_size,
    filename=weights_file)]      

# fine tune
model.fit_generator(generator=train_generator(test_df.iloc[train_idx]),
                    steps_per_epoch=np.ceil(float(len(train_idx)) / float(batch_size)),
                    epochs=50,
                    verbose=2,
                    callbacks=callbacks_list,
                    validation_data=valid_generator(test_df.iloc[valid_idx]),
                    validation_steps=np.ceil(float(len(valid_idx)) / float(batch_size)))

# evaluate predictions
min_loss = np.min(model.history.history['val_loss'])
max_metric = np.max(model.history.history['val_comp_metric_no_sigmoid'])
best_epoch = np.argmax(model.history.history['val_comp_metric_no_sigmoid'])

print(min_loss, max_metric, best_epoch)
loss_list.append(min_loss)
epoch_list.append(best_epoch)
metric_list.append(max_metric)

# log results
log_file = f'{FILE_PATH}/logs/{model_name}_{np.mean(max_metric):0.5f}.csv'
logger = pd.DataFrame({
    'backbone': 'backbone',    
    'decoder': "-".join(str(x) for x in decoder_filters),    
    'size': img_size,
    'cycle_len': cycle_len,
    'batch_size': batch_size,
    'mean_loss': np.mean(loss_list),
    'mean_metric': np.mean(metric_list),
    'best_epoch': np.median(epoch_list),},
    index=[0])
logger.to_csv(log_file, index=False)


Epoch 1/50
